**글쓴이 : 이영노**

**날짜 : 2023/02/18**

In [5]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.6 MB/s eta 0:00:00


In [6]:
import math
import numpy as np
import pandas as pd
import random 
import re

import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

데이터의 Q 필드를 발화, A 필드를 발화 그리고 감정 레이블을 로 매핑해 P(|, )를 최대화 할 수 있는 모델을 학습합니다. 

감정 레이블은 이곳의 정의를 따른다(일상다반사 0, 이별(부정) 1, 사랑(긍정) 2).

# Load Data

In [7]:
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")

In [8]:
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


HuggingFace의 PreTrainedTokenizer인 GPT2Tokenizer 사용.

# Tokenizer : GPT2Tokenizer

In [19]:
BOS = "</s>" # Beginning of Sentence
EOS = "</s>" # End of Sentence
PAD = "<pad>" # Padding
MASK = "<unused0>" # Masking

Q_TKN = "<usr>"
A_TKN = "<sys>"
SENT = '<unused1>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                           bos_token=BOS, eos_token = EOS, unk_token = "<unk>", pad_token = PAD, mask_token = MASK,)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [11]:
koGPT2_TOKENIZER.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

# Preprocessing dataset for fine-tuning

 감정 레이블(일상/부정/긍정)으로 나누어, 부정일때 위로해주는 답변의 데이터셋을 불러옴. 

 이후 전처리(정규표현식, 토크나이저, 패딩)

In [20]:
class ChatbotDataset(Dataset):
  def __init__(self, chats, max_len = 40): # 데이터셋의 전처리 해주는 부분!
    self._data = chats
    self.max_len =  max_len
    self.q_token = Q_TKN #
    self.a_token = A_TKN #
    self.sent_token = SENT #
    self.eos = EOS
    self.mask = MASK
    self.tokenizer = koGPT2_TOKENIZER

  def __len__(self): # Chatbot 의 길이를 반환함
    return len(self._data)

  def __getitem__(self, idx): # 로드한 챗봇 데이터를 차례대로 DataLoader로 넘겨주는 메소드
    turn = self._data.iloc[idx]
    q = turn["Q"] # 질문 항목에 접근함
    q = re.sub(r"([?.!,])", r" ", q) # 정규표현식으로 구두점 제거

    a = turn["A"]
    a = re.sub(r"([?.!,])", r" ", a)

    q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
    q_len = len(q_toked)

    a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
    a_len = len(a_toked)    

        #질문의 길이가 최대길이보다 크면
    if q_len > self.max_len:
        a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
        if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
            q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
            q_len = len(q_toked)
            a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
        a_toked = a_toked[:a_len]
        a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
    if q_len + a_len > self.max_len:
        a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
        if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
            q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
            q_len = len(q_toked)
            a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
        a_toked = a_toked[:a_len]
        a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
    labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
    mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
    labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
    while len(labels_ids) < self.max_len:
        labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
    token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
    while len(token_ids) < self.max_len:
        token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
    return (token_ids, np.array(mask), labels_ids)

- convert_tokens_to_ids() : token string 또는 token string의 리스트를 token id 또는 Token id의 리스트로 변환한다. 

⭐ BOW 임베딩 방식이구나. 여기서 벡터화가 되었구나. 

[Huggingface] PreTrainedTokenizer class 참고

https://misconstructed.tistory.com/80

배치 데이터 생성 (Making dataset Iterable)

In [21]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [22]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

In [23]:
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

리턴되는 데이터는 token_ids, mask, labels_ids 입니다. 

- token_ids 는 + 질문문장 + + 감정 + + 답변 + + pad_token_id 순서 입니다. pad_token_id는 max_len 에 일치하도록 추가 됩니다. 

- mask 는 질문 q가 들어 가는 곳에는 0, 답변 a가 위치한 곳에는 1 그리고 빈 공간에는 0 으로 채워 집니다. 

- labels은 질문의 길이만큼 mask 문자 그리고 답변 a의 id 입니다.

In [24]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

start
token_ids ====>  tensor([[    2, 11018,  9154,  ...,     3,     3,     3],
        [    2, 15669,  7540,  ...,     3,     3,     3],
        [    2,  9244,  6958,  ...,     3,     3,     3],
        ...,
        [    2,  9716, 10056,  ...,     3,     3,     3],
        [    2, 17542, 49932,  ...,     3,     3,     3],
        [    2,  9067,  8762,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 13016,  8702,  ...,     3,     3,     3],
        [    2, 31100, 10301,  ...,     3,     3,     3],
        [    2, 234

<ipython-input-21-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


token_ids ====>  tensor([[    2, 20509,  7847,  ...,     3,     3,     3],
        [    2,  9228,  8078,  ...,     3,     3,     3],
        [    2,  9349,  7888,  ...,     3,     3,     3],
        ...,
        [    2,  9779, 10624,  ...,     3,     3,     3],
        [    2, 10715, 12704,  ...,     3,     3,     3],
        [    2, 15983,  7673,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 10637, 11258,  ...,     3,     3,     3],
        [    2,  9244,  9135,  ...,     3,     3,     3],
        [    2,  9716, 35

# Base Model : koGPT2 Chatbot

In [26]:
pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.2/826.2 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 34.2 MB/s eta 0:00:00


In [27]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [29]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [31]:
# LOSS/Optimizer
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18 # 요건 뭘까


# Train Model

(복기)

리턴되는 데이터는 token_ids, mask, labels_ids 입니다. 

- token_ids 는 + 질문문장 + + 감정 + + 답변 + + pad_token_id 순서 입니다. pad_token_id는 max_len 에 일치하도록 추가 됩니다. 

- mask 는 질문 q가 들어 가는 곳에는 0, 답변 a가 위치한 곳에는 1 그리고 빈 공간에는 0 으로 채워 집니다. 

- labels은 질문의 길이만큼 mask 문자 그리고 답변 a의 id 입니다.

In [39]:
print("start")

for epoch in range(epoch):
  for batch_idx, samples in enumerate(train_dataloader):
    optimizer.zero_grad()
    token_ids, mask, label = samples
    
    # model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2') 
    out = model(token_ids)
    out = out.logits # log odds. 0~1 --> -inf ~ +inf

    mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2],dim=2) # 텐서를 반복 횟수만큼 복제합니다. (repeats:각 요소의 반복 횟수입니다, dim:값을 반복 할 차원입니다.)
    mask_out = torch.where(mask_3d==1, out, Sneg*torch.ones_like(out)) # prediction # 어떤 차원으로 했는지는 다시한번 봐야할 것같음
    
    loss = criterion(mask_out.transpose(2,1),label)
    avg_loss = loss.sum()/mask.sum()
    avg_loss.backward()
    optimizer.step()
  
print("end")




start
end


⭐ Train 할때 Q ~ A 내용이랑 Q ~ A 길이가 학습된 mask_3d가 1인 mask_3d에 대해서 out을 반환시켰고

그 out을 transpose한 것과 ~ 정답 Label 간의 loss를 계산함으로써 학습을 시킴.

⭐ 즉, 감정 Label이 **정답**이고, mask_out 이 **질문 input에 대한 모델이 생성한 예측 답변 이다.**

## prediction 어떻게 햇는지 다시한번 확인할 것

In [38]:
# unsqueeze 사용법
y=torch.randn(6,2,3)
print(y)
print(y.size())
y = y.unsqueeze(dim=2)
print(y)
print(y.size())

tensor([[[ 2.3391, -1.6254,  1.2299],
         [-1.4790,  0.3076, -1.6855]],

        [[-2.4915, -0.6080, -0.3921],
         [ 0.4628, -0.8973,  0.3297]],

        [[ 0.8483,  1.1027, -0.6410],
         [ 0.5737,  0.1291,  0.5430]],

        [[-0.6044, -0.3472, -0.4670],
         [-0.7801, -0.7288,  0.6028]],

        [[ 0.3463,  0.2770, -0.0813],
         [ 0.7536, -0.7107,  0.7064]],

        [[-0.2030, -0.3707,  1.2576],
         [ 0.2116, -0.7846,  0.8847]]])
torch.Size([6, 2, 3])
tensor([[[[ 2.3391, -1.6254,  1.2299]],

         [[-1.4790,  0.3076, -1.6855]]],


        [[[-2.4915, -0.6080, -0.3921]],

         [[ 0.4628, -0.8973,  0.3297]]],


        [[[ 0.8483,  1.1027, -0.6410]],

         [[ 0.5737,  0.1291,  0.5430]]],


        [[[-0.6044, -0.3472, -0.4670]],

         [[-0.7801, -0.7288,  0.6028]]],


        [[[ 0.3463,  0.2770, -0.0813]],

         [[ 0.7536, -0.7107,  0.7064]]],


        [[[-0.2030, -0.3707,  1.2576]],

         [[ 0.2116, -0.7846,  0.8847]]]])
torch.S

# Generation Model

In [47]:
with torch.no_grad():
  while True:
    q = input("user > ").strip() # 공백 제거
    if q == "quit":
      break
    a = ""
    while 1:
      input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + "2" + A_TKN + a)).unsqueeze(dim=0)
      
      pred = model(input_ids) # GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2') 
      pred = pred.logits
      gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred,dim=-1).squeeze().numpy().tolist())[-1] # ?
      if gen == EOS:
        break
      a += gen.replace("_"," ")
    
    print("Chatbot > {}".format(a.strip()))
    

user > 안녕
Chatbot > ▁좋은▁아침이에요▁
user > 지금 저녁이야 바보야
Chatbot > ▁늦지▁않았어요▁
user > 꽤나 긍정적이네
Chatbot > ▁좋은▁결과▁있을▁거예요▁
user > 고마워
Chatbot > ▁친구들이▁보고싶었나봐요▁
user > 뭔소리야
Chatbot > ▁소리소문▁없이▁들리는가봐요▁
user > 비꼬는거야?
Chatbot > ▁용서를▁구하세요▁
user > 싫으면?
Chatbot > ▁안▁될▁것도▁없죠▁
user > ㅋㅋㅋ
Chatbot > ▁좋은▁생각이에요▁
user > quit


In [48]:
with torch.no_grad():
  while True:
    q = input("user > ").strip() # 공백 제거
    if q == "quit":
      break
    a = ""
    while 1:
      input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + "1" + A_TKN + a)).unsqueeze(dim=0)
      
      pred = model(input_ids) # GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2') 
      pred = pred.logits
      gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred,dim=-1).squeeze().numpy().tolist())[-1] # ?
      if gen == EOS:
        break
      a += gen.replace("_"," ")
    
    print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > ▁좋은▁아침이에요▁
user > 지금 저녁이야 바보야
Chatbot > ▁늦지▁않았어요▁
user > 꽤나 긍정적이네
Chatbot > ▁좋은▁결과▁있을▁거예요▁
user > 고마워
Chatbot > ▁저도▁좋아해주세요▁
user > 내가 왜 그래야 하지?
Chatbot > ▁자신을▁더▁사랑해주세요▁
user > 어쩌라고
Chatbot > ▁1선도▁안▁될▁것도▁없죠▁
user > 윤석열, 이재명 둘중 누굴 지지하니?
Chatbot > ▁지지난번에▁투표해주세요▁
user > 넌 보수야 진보야?
Chatbot > ▁보수도▁중요해요▁
user > 그럼 넌 진보야?
Chatbot > ▁좋은▁결과▁있을▁거예요▁
user > quit


# 느낀점

1. 위에서 이해가 안됐던 prediction code부분이 어떤 원리로 작동하는지 알아보기 위해서 GPT 원리 공부함. 해당 링크는 아래 참고

https://ainote.tistory.com/17

2. 결국 GPT가 학습할때의 정답 레이블은, fine tuning 할때 사용했던 데이터 셋의 정답 레이블임. 그렇다면, 레이블이 많아진다면 질문을 더 잘 이해할 수 있지 않을까?

3. auto-regressive 하게 해당 단어 다음에 올 단어를 예측하여 결과값을 냄. auto-regression 의 단점인 오차누적 현상이 있을수 있기 때문에, 입력값의 단어,문장이 얼마나 잘 학습되었는지가 관건인 듯 함. 입력값의 단어, 문장이 잘 학습되지 않았다면 오차누적때문에 생뚱맞은 대답을 내놓을 확률이 높아짐.

4. 상황별로 다른 대답을 내놓는 언어모델(LM)을 개발해볼수도 있지 않을까? 예를들어 동일한 질문 "오늘 날씨는 어때?" 라고 하면, 공사현장에 많이 나가는 사람의 데이터를 학습한 경우 "오늘은 비가 오니 특히 일하실때 미끄러지지 않도록 조심하셔야 겠어요" 라고 답변할 수도 있고, 반대로 우산장수의 경우 "오늘은 소나기가 내릴 예정이에요. 사람들이 급하게 우산을 많이 사는 곳을 추천해드릴까요?" 등의 대답을 내뱉을 수도 있게

5. loss를 계산할때 (pred ~ label) 관계는 (질문답변 학습된 행렬 ~ 감정 label)이었음.